In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 유방암 데이터셋

## 유방암 진단 데이터

In [36]:
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'
df = pd.read_csv(path, header=None)
df.head() #32열(샘플ID, 진단결과, 특성1, 특성2, ....)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


- 0번째 칼럼 : ID
- 1번째 칼럼 : 타겟
    - 악성 : M / 양성 : B
- 2번째 칼럼~ : 특성

In [37]:
# X, y
X = df.iloc[:, 2:]
y = df.iloc[:, 1]
y = np.where(y == 'M', 1, 0)

In [38]:
# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

# 파이프 라인

## 파이프라인을 사용한 워크플로
- 전처리, 학습, 예측 단계를 연결하여 효율적으로 워크플로를 관리하는 방법
- 여러 개의 사이킷런 변환기와 추정기를 연결
    - 변환기 : transform 메서드를 가진
    - 추정기 : predict 메서드를 가진
- sklearn.pipeline.make_pipeline

In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(StandardScaler(),
                         PCA(n_components = 2),
                         LogisticRegression())

pipeline.fit(X_train, y_train)  # 변환기들의 fit & transform -> 추정기의 fit 호출

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=2)),
                ('logisticregression', LogisticRegression())])

In [40]:
pipeline.predict(X_test)  # 변환기들의 tranform -> 추정기의 predict 호출

array([1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 0])

In [41]:
pipeline.score(X_test, y_test)

0.9736842105263158

In [42]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pipeline.predict(X_test))

array([[70,  2],
       [ 1, 41]], dtype=int64)

# 모델 튜닝과 교차검증

## 모델(모형) 튜닝
- 모형의 성능은 데이터와 초모수(모형의 설정 인수)에 영향을 받음
- 모델 훈련(학습) : 데이터로부터 모형의 모수(파라미터)를 추정하는 과정
- 모델 튜닝(선택) : 모형의 초모수(하이퍼파라미터)를 조정하는 과정
    - ex) SVC : c, gamma
    - 여러 값을 시도해보고 가장 좋은 모형을 선택해야 함
    - 별도의 검증용 데이터를 사용하거나 교차검증

## 교차검증(CV : Cross-Validation)
- 데이터가 충분치 않아 검증용 데이터를 따로 할당하기 어려운 경우 사용
- 훈련 데이터를 여러 번 반복해서 나누고 여러 모델을 학습하고 평가하는 방법
- 대표적인 방법 : k-겹 교차검증(k-fold CV)
    - 전체 데이터를 K등분하여 K-1개 그룹은 훈련용으로, 나머지 한 그룹을 검증용으로 사용하는 과정을 반복하며 평균을 구함(일반적으로 K = 10 사용)
    - EX) <img src = 'https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F3gQO8%2FbtqF0ZOHja8%2FSUTbGTYwVndcUJ5qWusqa0%2Fimg.png' width = '600' />
1. sklearn.model_selection.cross_val_score
    - cv : k 값 설정
    - n_jobs : 사용할 코어의 수(병렬), -1은 모든 코어를 사용함
2. sklearn.model_selection.GridSearchCV
    - 리스트로 지정된 여러 가지 하이퍼파라미터 값의 모든 가능한 조합에 대해 모델 성능을 평가하여 최적의 조합을 찾음

In [43]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(estimator = pipeline, X = X_train, y = y_train, 
                       cv = 10, n_jobs = -1)
np.mean(score)

0.9470531400966185

유방암 데이터에 대한 결정트리 모형 튜닝 
- cross_val_score 사용

In [44]:
from sklearn.tree import DecisionTreeClassifier

best_score = 0
for k in range(1, 11):
    scores_tree = cross_val_score(
        estimator = DecisionTreeClassifier(max_depth = k),
        X = X_train, y = y_train, cv = 10, n_jobs = -1)
    score = np.mean(scores_tree)
    
    print('depth = ', k, 'accuracy = ', score)
    if score > best_score:
        best_score = score
        best_parameters = {'max_depth' : k} 

print()
print(best_score)
print(best_parameters)

depth =  1 accuracy =  0.9010628019323672
depth =  2 accuracy =  0.9186473429951691
depth =  3 accuracy =  0.9208212560386473
depth =  4 accuracy =  0.9253140096618357
depth =  5 accuracy =  0.9341545893719807
depth =  6 accuracy =  0.927536231884058
depth =  7 accuracy =  0.927536231884058
depth =  8 accuracy =  0.9187439613526569
depth =  9 accuracy =  0.9340579710144927
depth =  10 accuracy =  0.9229951690821256

0.9341545893719807
{'max_depth': 5}


In [45]:
tree = DecisionTreeClassifier(**best_parameters)  # **kwargs : 키워드 인자(딕셔너리)
tree.fit(X_train, y_train)
tree.score(X_test, y_test)

0.9473684210526315

유방암 데이터에 대한 결정트리 모형 튜닝 
- GridSearchCV 사용

In [46]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth' : range(1, 11)}  # 딕셔너리
gscv = GridSearchCV(estimator = DecisionTreeClassifier(),  # 추정기 초모수에 넣는 것이 아니라 param_grid 파라미터에 넣어야 함
                    param_grid = param_grid,  # 요기
                    scoring = 'accuracy', cv = 10, n_jobs = -1)  

gscv.fit(X_train, y_train)
print(gscv.best_score_)
print(gscv.best_params_)

0.9428985507246377
{'max_depth': 5}


유방암 데이터에 대한 SVC 모형 튜닝 
- GridSearchCV 사용

In [47]:
from sklearn.svm import SVC

pipe_svc = make_pipeline(StandardScaler(), SVC(kernel='rbf'))

p_list = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]  # 로그 스케일
param_grid = {'svc__C' : p_list, 'svc__gamma' : p_list}  # value는 리스트 형태
gs = GridSearchCV(estimator = pipe_svc,  # estibator에 파이프라인
                  param_grid=param_grid,
                  scoring = 'accuracy', cv = 10, n_jobs = -1)

gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)
print(gs.score(X_test, y_test))

0.9801449275362317
{'svc__C': 10.0, 'svc__gamma': 0.01}
0.9912280701754386
